In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import random
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
train_path = "../input/cat-and-dog/training_set/training_set/"
train_path_animal = [train_path + i + "/" for i in os.listdir(train_path)]

In [ ]:
def read_image(image_path):
    img = cv2.imread(image_path)
    return cv2.resize(img, (64, 64))

In [ ]:
x, y, label = [], [], 0
transform=transforms.Compose([transforms.Resize((64,64))])
for i in train_path_animal:
    for j in os.listdir(i):
        if random.random() > 0.05:continue
        if j[-1] == 'e':continue
        img = np.array(read_image(i+j))
        x.append(img)
        y.append(label)
    label += 1
X_train, Y_train, X_test, Y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
x_train, x_test, y_train, y_test = np.array(X_train), np.array(Y_train), np.array([X_test]), np.array([Y_test])
x_train.shape, x_test.shape,y_train.shape, y_test.shape

((313, 64, 64, 3), (79, 64, 64, 3), (1, 313), (1, 79))

In [ ]:
def sigmoid(Z):
    return 1/(1+np.exp(-Z)), Z

def relu(Z):
    A = np.maximum(0,Z)
    cache = Z 
    return np.maximum(0,Z), Z

def relu_backward(dA, cache):
    dZ = np.array(dA, copy=True)
    dZ[cache <= 0] = 0
    return dZ

def sigmoid_backward(dA, cache):
    s = 1/(1+np.exp(-cache))
    dZ = dA * s * (1-s)
    return dZ

In [ ]:
# Reshape the training and test examples 
train_x_flatten = x_train.reshape(x_train.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = x_test.reshape(x_test.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
x_train = train_x_flatten/255.
x_test = test_x_flatten/255.

print ("train_x's shape: " + str(x_train.shape))
print ("test_x's shape: " + str(x_test.shape))

train_x's shape: (12288, 313)
test_x's shape: (12288, 79)


In [ ]:
def linear_forward(A, W, b):
    Z = np.dot(W,A)+b
    cache = (A, W, b)
    return Z, cache

def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db

def linear_activation_forward(A_prev, W, b, activation):
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z)
    cache = (linear_cache, activation_cache)
    return A, cache

def linear_activation_backward(dA, cache, activation):
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
    return dA_prev, dW, db

In [ ]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    return parameters

def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)]-learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] =  parameters["b" + str(l+1)]-learning_rate*grads["db"+str(l+1)]
    ### END CODE HERE ###
    return parameters

In [ ]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev,parameters["W"+str(l)],parameters["b"+str(l)],"relu")
        caches.append(cache)
        None
    AL, cache = linear_activation_forward(A,parameters["W"+str(L)],parameters["b"+str(L)],"sigmoid")
    caches.append(cache)
    return AL, caches

# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dAL = -(np.divide(Y,AL)-np.divide(1-Y,1-AL))

    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL,current_cache,"sigmoid")
   
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA"+str(l+1)],current_cache,"relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [ ]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]
    cost = (-1/m)*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    cost = np.squeeze(cost)
    
    return cost

In [ ]:
x_train.shape, x_test.shape,y_train.shape, y_test.shape

((12288, 313), (12288, 79), (1, 313), (1, 79))

In [ ]:
# GRADED FUNCTION: L_layer_model

def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 2500, print_cost=True):
    
    costs = []                         # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    
    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X,parameters)

        cost = compute_cost(AL,Y)

        grads = L_model_backward(AL,Y,caches)

        parameters = update_parameters(parameters,grads,learning_rate)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        if print_cost and i % 100 == 0:
            costs.append(cost)
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return parameters

In [ ]:
layers_dims = [12288, 20, 15, 7, 5, 1] #  4-layer model
model=L_layer_model(x_train,y_train,layers_dims)

Cost after iteration 0: 0.693147


In [ ]:
print(model)

{'W1': array([[ 0.01788628,  0.0043651 ,  0.00096497, ...,  0.00742033,
         0.00777721, -0.02044101],
       [-0.02034742, -0.01277108, -0.00845047, ..., -0.01592859,
         0.01189758,  0.01369089],
       [ 0.00736324,  0.01040033, -0.00610758, ..., -0.00719971,
         0.01342522, -0.00194118],
       ...,
       [ 0.00152688,  0.01171849, -0.01256989, ..., -0.01793974,
         0.00977006,  0.00740466],
       [ 0.00301225,  0.01519223,  0.00774002, ..., -0.000818  ,
        -0.00483844,  0.01257786],
       [ 0.01000492,  0.0052482 , -0.00076459, ...,  0.00668238,
         0.00346637, -0.0061899 ]]), 'b1': array([[ 0.00000000e+00],
       [-1.62480444e-08],
       [ 7.65075899e-09],
       [ 1.62407125e-09],
       [-3.84979802e-09],
       [ 3.60901851e-09],
       [ 7.82870448e-09],
       [-5.86674897e-11],
       [-1.35419800e-09],
       [ 1.67659674e-09],
       [-4.18583417e-09],
       [ 1.51785078e-09],
       [-4.66912535e-09],
       [ 2.11961873e-09],
       [ 